In [44]:
import random
from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END

In [45]:
class AgentState(TypedDict):
    name: str
    guesses: List[int]
    attempts: int
    lower_bound: int
    upper_bound: int

In [55]:
MAX_GUESSES = 5

SETUP = "SETUP"
GUESS = "GUESS"
HINT = "HINT"
LOOP = "continue"
EXIT = "exit"

In [ ]:
def setup(state: AgentState) -> AgentState:
    """Sets up all the initial values in the state"""

    state['attempts'] = 0
    state['guesses'] = []
    state['lower_bound'] = 1
    state['upper_bound'] = 20

    print(f"\n{state["name"]}, the target is set to {TARGET}. Lets roll!")
    return state

def guess(state: AgentState) -> AgentState:
    """Guess a number within the given lower and upper bounds"""
    num = random.randint(state["lower_bound"], state["upper_bound"])
    state["guesses"].append(num)
    state['attempts'] += 1

    print(f"Guessed {num} from [{state["lower_bound"]}, {state["upper_bound"]}]")
    return state

def hint(state: AgentState) -> AgentState:
    """Validate the guessed value and give a hint if that's wrong"""
    guess = state["guesses"][-1]
    if guess < TARGET:
        state["lower_bound"] = guess + 1
        print("HINT: Higher")
    if guess > TARGET:
        state["upper_bound"] = guess - 1
        print("HINT: Lower")
    return state

def should_continue(state: AgentState) -> AgentState:
    """Decides if the game should continue"""

    guess = state["guesses"][-1]
    if guess == TARGET:
        print(f"Amazing! The guess {guess} was right. You have guessed it in {state["attempts"]} attempts")
        return EXIT
    elif state['attempts'] == MAX_GUESSES:
        print(f"You have exhausted all your attempts before guessing the number {TARGET}. Better luck next time!")
        return EXIT
    else:
        return LOOP

In [51]:
graph = StateGraph(AgentState)

graph.add_node(SETUP, setup)
graph.add_node(GUESS, guess)
graph.add_node(HINT, hint)

graph.add_edge(START, SETUP)
graph.add_edge(SETUP, GUESS)
graph.add_edge(GUESS, HINT)
graph.add_conditional_edges(HINT, should_continue, {
    LOOP: GUESS,
    EXIT: END
})

app = graph.compile()

In [60]:
TARGET = random.randint(1, 20)

input_state = AgentState({ "name": "Sudeep" })

result = app.invoke(input_state)


Sudeep, the target is set to 2. Lets roll!
Guessed 16 from 1, 20
HINT: Lower
Guessed 11 from 1, 15
HINT: Lower
Guessed 1 from 1, 10
HINT: Higher
Guessed 3 from 2, 10
HINT: Lower
Guessed 2 from 2, 2
Amazing! The guess 2 was right. You have guessed it in 5 attempts


In [50]:
# from IPython.display import Image, display

# display(Image(app.get_graph().draw_mermaid_png()))